https://www.kaggle.com/code/faressayah/lending-club-loan-defaulters-prediction

# 💾 Data Description

- loan_amnt             
  - L'importo indicato del prestito richiesto dal richiedente. Se in un certo momento il dipartimento crediti riduce l'importo del prestito, ciò sarà riflesso in questo valore.
- term                  
  - Il numero di pagamenti del prestito. I valori sono in mesi e possono essere sia 36 che 60.
- int_rate              
  - Tasso di interesse sul prestito
- installment           
  - Il pagamento mensile dovuto dal richiedente se il prestito è erogato.
- grade                 
  - Grado di prestito assegnato da LC
- sub_grade             
  - Sotto-grado di prestito assegnato da LC
- emp_title             
  - Il titolo di lavoro fornito dal richiedente al momento della richiesta del prestito.
- emp_length            
  - Durata dell'occupazione in anni. I valori possibili sono compresi tra 0 e 10 dove 0 significa meno di un anno e 10 significa dieci o più anni.
- home_ownership        
  - Lo stato di proprietà della casa fornito dal richiedente durante la registrazione o ottenuto dal rapporto di credito. I nostri valori sono: AFFITTO, PROPRIETARIO, MUTUO IPOTECARIO, ALTRO
- annual_inc            
  - Il reddito annuale autodichiarato fornito dal richiedente durante la registrazione.
- verification_status   
  - Indica se il reddito è stato verificato da LC, non verificato o se la fonte del reddito è stata verificata
- issue_d               
  - Il mese in cui il prestito è stato finanziato
- loan_status           
  - Stato attuale del prestito
- purpose               
  - Una categoria fornita dal richiedente per la richiesta di prestito.
- title                 
  - Il titolo del prestito fornito dal richiedente
- zip_code              
  - I primi 3 numeri del codice postale forniti dal richiedente nella domanda di prestito.
- addr_state            
  - Lo stato fornito dal richiedente nella domanda di prestito
- dti                   
  - Un rapporto calcolato utilizzando i pagamenti mensili totali del debito del richiedente sul totale degli obblighi di debito, escludendo il mutuo e il prestito LC richiesto, diviso per il reddito mensile autodichiarato del richiedente.
- earliest_cr_line      
  - Il mese in cui è stata aperta la prima linea di credito segnalata del richiedente
- open_acc              
  - Il numero di linee di credito aperte nel file di credito del richiedente.
- pub_rec               
  - Numero di record pubblici negativi
- revol_bal             
  - Bilancio totale del credito rotativo
- revol_util            
  - Tasso di utilizzo della linea di credito rotativo, o l'importo di credito utilizzato dal richiedente rispetto al totale del credito rotativo disponibile.
- total_acc             
  - Il numero totale di linee di credito attualmente nel file di credito del richiedente
- initial_list_status   
  - Lo stato iniziale dell'elenco del prestito. I valori possibili sono - W, F
- application_type      
  - Indica se il prestito è una domanda individuale o una domanda congiunta con due co-richiedenti
- mort_acc              
  - Numero di conti ipotecari.
- pub_rec_bankruptcies  
  - Numero di fallimenti registrati pubblicamente


In [55]:
!pip install -q hvplot

In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats 
import matplotlib.pyplot as plt
import hvplot.pandas

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import (
    accuracy_score, confusion_matrix, classification_report, 
    roc_auc_score, roc_curve, auc
)
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay

from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

pd.set_option('display.float', '{:.2f}'.format)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 50)

In [57]:
data = pd.read_csv("C:/Users/edoardo.frapiccini/Desktop/vai/AI/pratica/Machine learning for credit scoring/progetti/Lending Club Loan Defaulters Prediction/dataset/Lending Club Dataset/lending_club_loan_two.csv")

#elimino tutti i valori mancanti
data.dropna(inplace=True)

#elimino le colonne che non mi servono
data.drop('emp_length', axis=1, inplace=True)
data.drop('title', axis=1, inplace=True)
data.drop('grade', axis=1, inplace=True)
data.drop('issue_d', axis=1, inplace=True)
#data.drop('loan_amnt', axis=1, inplace=True) #!!!

#converto i valori categorici in numerici
""" data.drop('emp_title', axis=1, inplace=True) #troppe categorie diverse """
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data['emp_title'] = le.fit_transform(data['emp_title'])

term_values = {' 36 months': 36, ' 60 months': 60}
data['term'] = data.term.map(term_values)

data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])
data['earliest_cr_line'] = data.earliest_cr_line.dt.year

""" data['zip_code'] = data.address.apply(lambda x: x[-5:])
data = pd.get_dummies(data, columns=['zip_code'], drop_first=True) """
data.drop('address', axis=1, inplace=True)

dummies = ['loan_status', 'sub_grade', 'verification_status', 'purpose', 'initial_list_status', 
           'application_type', 'home_ownership']
data = pd.get_dummies(data, columns=dummies, drop_first=True)

bool_columns = data.select_dtypes(include='bool').columns
data[bool_columns] = data[bool_columns].astype(int)

# Calcola la correlazione rispetto alla colonna 'loan_status_Fully Paid'
correlation = data.corr()['loan_status_Fully Paid'].drop('loan_status_Fully Paid')

condition_to_view = ((correlation < -0.05) | (correlation > 0.05)) & (correlation.index != 'installment')

columns_to_view = correlation.loc[condition_to_view].to_frame()

#visualizzazione della correlazione
columns_to_view.hvplot.barh(
    width=600, height=1200, 
    title="Correlation between Loan status and Numeric Features", 
    ylabel='Correlation', xlabel='Numerical Features', 
)


C:\Users\edoardo.frapiccini\AppData\Local\Temp\ipykernel_10368\2773865807.py:22: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data['earliest_cr_line'] = pd.to_datetime(data['earliest_cr_line'])


:Bars   [index]   (loan_status_Fully Paid)

In [58]:
# Trova le colonne con correlazione compresa tra 0.049 e -0.049
columns_to_drop = correlation.loc[((correlation > -0.049) & (correlation < 0.049)) & (correlation.index != 'installment')].index

# Elimina le colonne dal DataFrame
data.drop(columns=columns_to_drop, inplace=True)

sub_grade_columns = data.filter(like='sub_grade').columns
data.drop(columns=sub_grade_columns, inplace=True)

#prendo solo i valori di loan_status_Fully Paid = 1
data_1 = data[data['loan_status_Fully Paid'] == 1]
data_0 = data[data['loan_status_Fully Paid'] == 0]

#splitto i dati in train e test
train, test = train_test_split(data_1, test_size=0.33, random_state=42)

#rimuovo i valori anomali
train = train[train['annual_inc'] <= 250000]
train = train[train['dti'] <= 50]

X_train, y_train = train.drop('installment', axis=1), train.installment
X_test, y_test = test.drop('installment', axis=1), test.installment

""" scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  """

' scaler = MinMaxScaler()\nX_train = scaler.fit_transform(X_train)\nX_test = scaler.transform(X_test)  '

RANDOM FOREST

In [68]:
""" from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Definire i parametri da testare
parametri = {
    'n_estimators': [100,200],
    'max_depth': [3,4,6],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Creare il regressore RandomForest
rf_regressor = RandomForestRegressor(random_state=42)

# Creare l'oggetto GridSearchCV
grid_search = GridSearchCV(estimator=rf_regressor, param_grid=parametri, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)

# Eseguire la ricerca dei parametri migliori
grid_search.fit(X_train, y_train)

# Stampare i migliori parametri trovati
print("Migliori parametri:", grid_search.best_params_)

# Stampare il miglior punteggio (negative mean squared error)
print("Miglior punteggio:", grid_search.best_score_) """

Migliori parametri: {'max_depth': 6, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Miglior punteggio: -425.36745034236816


In [69]:
from sklearn.ensemble import RandomForestRegressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=6, min_samples_split=2, min_samples_leaf=1, random_state=42)

rf_regressor.fit(X_train, y_train)
# Predizione
y_test_pred = rf_regressor.predict(X_test)
results = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_test_pred})
print(results.head(10))

        True Values  Predicted Values
294820       320.95            321.80
304941       330.73            335.06
378279       244.30            255.47
326739       460.56            488.31
66338        506.34            470.54
386063       505.34            533.93
328847       614.08            634.76
44831        178.24            170.59
273144       138.65            136.96
208423        55.54             42.74


In [70]:
X_test_2, y_test_2 = data_0.drop('installment', axis=1), data_0.installment
""" X_test_2 = scaler.transform(X_test_2) """

y_test_pred_2 = rf_regressor.predict(X_test_2)

results = pd.DataFrame({'True Values': y_test_2, 'Predicted Values': y_test_pred_2})
print(results.head(10))

    True Values  Predicted Values
4        609.33            617.86
26       284.45            281.14
48       411.01            430.17
54       244.36            277.89
69       117.14            104.65
72       271.27            256.51
81       356.48            335.06
82       285.21            281.60
84       267.47            256.51
97       399.86            380.96


XG BOOST

In [59]:
""" from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
import xgboost as xgb 
gbm = xgb.XGBRegressor()
reg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]
                            ,'max_depth': [3,4,6], 'n_estimators': [500,1000]}, verbose=1)
reg_cv.fit(X_train,y_train)
best_params = reg_cv.best_params_
print(f"Best Parameters: {best_params}") """

' from sklearn.model_selection import RandomizedSearchCV,GridSearchCV\nimport xgboost as xgb \ngbm = xgb.XGBRegressor()\nreg_cv = GridSearchCV(gbm, {"colsample_bytree":[1.0],"min_child_weight":[1.0,1.2]\n                            ,\'max_depth\': [3,4,6], \'n_estimators\': [500,1000]}, verbose=1)\nreg_cv.fit(X_train,y_train)\nbest_params = reg_cv.best_params_\nprint(f"Best Parameters: {best_params}") '

In [60]:
import xgboost as xgb
gbm = xgb.XGBRegressor(colsample_bytree=1.0, max_depth=3, min_child_weight=1.2, n_estimators=500)

gbm.fit(X_train, y_train)
# Predizione
y_test_pred = gbm.predict(X_test)
results = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_test_pred})
print(results.head(10))

        True Values  Predicted Values
294820       320.95            323.93
304941       330.73            329.60
378279       244.30            245.07
326739       460.56            459.60
66338        506.34            507.70
386063       505.34            504.94
328847       614.08            614.40
44831        178.24            179.73
273144       138.65            139.86
208423        55.54             58.63


In [61]:
X_test_2, y_test_2 = data_0.drop('installment', axis=1), data_0.installment
""" X_test_2 = scaler.transform(X_test_2) """

y_test_pred_2 = gbm.predict(X_test_2)

results = pd.DataFrame({'True Values': y_test_2, 'Predicted Values': y_test_pred_2})
print(results.head(10))

    True Values  Predicted Values
4        609.33            602.82
26       284.45            285.56
48       411.01            416.01
54       244.36            241.26
69       117.14            115.56
72       271.27            271.01
81       356.48            357.60
82       285.21            283.92
84       267.47            270.19
97       399.86            404.61


REGRESSIONE POLINOMIALE

In [62]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler
from sklearn.linear_model import Lasso

# Definisco il modello con regressione polinomiale e penalizzazione Lasso
model = Pipeline([
    ('poly_features', PolynomialFeatures(degree=2)),
    ('lasso', Lasso(alpha=1))
])

# Addestramento del modello
model.fit(X_train, y_train)
# Predizione
y_test_pred = model.predict(X_test)
results = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_test_pred})
print(results.head(10))

        True Values  Predicted Values
294820       320.95            321.63
304941       330.73            331.70
378279       244.30            244.90
326739       460.56            459.73
66338        506.34            508.43
386063       505.34            505.50
328847       614.08            611.72
44831        178.24            179.36
273144       138.65            139.42
208423        55.54             53.95


In [63]:
X_test_2, y_test_2 = data_0.drop('installment', axis=1), data_0.installment
""" X_test_2 = scaler.transform(X_test_2) """

y_test_pred_2 = model.predict(X_test_2)

results = pd.DataFrame({'True Values': y_test_2, 'Predicted Values': y_test_pred_2})
print(results.head(10))

    True Values  Predicted Values
4        609.33            256.31
26       284.45            161.45
48       411.01            178.61
54       244.36             66.33
69       117.14             67.67
72       271.27            167.03
81       356.48            211.48
82       285.21            168.17
84       267.47            161.64
97       399.86            199.24


ANN

In [ ]:
""" import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers

# define regression model
def regression_model():
    # create model
    ann = Sequential()
    #ann.add(Dense(150, activation='relu', input_shape=(X_train.shape[1],))) 
    #ann.add(Dense(150, activation='relu'))
    ann.add(Dense(150, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l1(1.0))) 
    ann.add(Dense(150, activation='relu', kernel_regularizer=regularizers.l1(1.0)))
    ann.add(Dense(1))
    
    # compile model
    ann.compile(optimizer=Adam(1e-3), loss='mean_squared_error') 
    #l'ottimizzatore Adam è un algoritmo di ottimizzazione che può essere utilizzato invece del procedimento di discesa del gradiente stocastico

    return ann

ann = regression_model()
ann.fit(X_train, y_train, validation_split=0.3, epochs=20)

# Predizione
y_test_ann = ann.predict(X_test)
y_test_ann = y_test_ann.reshape(-1)
results_ann = pd.DataFrame({'True Values': y_test, 'Predicted Values': y_test_ann})
print(results_ann.head(10)) """

In [ ]:
""" y_test_ann_2 = ann.predict(X_test_2)
y_test_ann_2 = y_test_ann_2.reshape(-1)
results_ann = pd.DataFrame({'True Values': y_test_2, 'Predicted Values': y_test_ann_2})
print(results_ann.head(10)) """